In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ship = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='ship')
df_ship_distance = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='ship_distance')
df_asj = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='ASJ')
df_ren = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='REN')
df_aka = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='AKA')
df_fwcost = pd.read_excel('../data/freshwater_datamaster.xlsx', sheet_name='fw_cost')


In [3]:
df_ship

,ship,tank_capacity,gross tonnage,deadweight tonnage
0,ASJ,28000,6093,8528
1,REN,25000,5823,8318
2,AKA,27000,5501,8037


In [4]:
df_asj

,VOYAGE,PORTS,TRIPS
0,1-23,SURABAYA;BALIKPAPAN;PALU,SURABAYA-BALIKPAPAN;BALIKPAPAN-PALU;PALU-SURABAYA
1,2-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...
2,3-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...
3,4-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...
4,5-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...
5,6-23,SURABAYA;MAKASSAR;KENDARI;BAUBAU,SURABAYA-MAKASSAR;MAKASSAR-KENDARI;KENDARI-BAU...
6,7-23,SURABAYA;MAKASSAR;KENDARI;BAUBAU,SURABAYA-MAKASSAR;MAKASSAR-KENDARI;KENDARI-BAU...
7,8-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...
8,9-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...
9,10-23,SURABAYA;MAKASSAR;BAUBAU;KENDARI,SURABAYA-MAKASSAR;MAKASSAR-BAUBAU;BAUBAU-KENDA...


In [5]:
df_ren

,VOYAGE,PORTS,TRIPS
0,1-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
1,2-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
2,3-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
3,4-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
4,5-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
5,6-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
6,7-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
7,8-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
8,9-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA
9,10-24,JAKARTA;PONTIANAK,JAKARTA-PONTIANAK;PONTIANAK-JAKARTA


In [6]:
df_aka

,VOYAGE,PORTS,TRIPS
0,1-24,SURABAYA;SAMARINDA;BOMBANA;MAKASSAR,SURABAYA-SAMARINDA;SAMARINDA-BOMBANA;BOMBANA-M...
1,2-24,MAKASSAR;BALIKPAPAN;SAMARINDA;BOMBANA;SAMARINDA,MAKASSAR-BALIKPAPAN;BALIKPAPAN-SAMARINDA;SAMAR...
2,3-24,SAMARINDA;BANJARMASIN;SURABAYA,SAMARINDA-BANJARMASIN;BANJARMASIN-SURABAYA;SUR...
3,4-24,SURABAYA;BALIKPAPAN,SURABAYA-BALIKPAPAN;BALIKPAPAN-SURABAYA
4,5-24,SURABAYA;SAMARINDA,SURABAYA-SAMARINDA;SAMARINDA-SURABAYA
5,6-24,SURABAYA;BALIKPAPAN;PALU,SURABAYA-BALIKPAPAN;BALIKPAPAN-PALU;PALU-SURABAYA
6,7-24,SURABAYA;MAKASSAR;SAMARINDA,SURABAYA-MAKASSAR;MAKASSAR-SAMARINDA;SAMARINDA...
7,8-24,SURABAYA;BALIKPAPAN,SURABAYA-BALIKPAPAN;BALIKPAPAN-SURABAYA
8,9-24,SURABAYA;BALIKPAPAN;KENDARI,SURABAYA-BALIKPAPAN;BALIKPAPAN-KENDARI;KENDARI...
9,10-24,SURABAYA;KENDARI,SURABAYA-KENDARI;KENDARI-SURABAYA


In [59]:
ships = df_ship['ship'].to_list()
ships

['ASJ', 'REN', 'AKA']

In [60]:
df_ship.loc[df_ship['ship'] == 'REN'].values[0]

array(['REN', 25000, 5823, 8318], dtype=object)

In [69]:
filtered_ports = df_aka.loc[df_aka["VOYAGE"] == "1-24", "PORTS"]

if not filtered_ports.empty:
    ports = filtered_ports.iloc[0]  
else:
    ports = None  

print(ports)

SURABAYA;SAMARINDA;BOMBANA;MAKASSAR


In [70]:
def get_list(df, reference, reference_column, destination_column):
    lst = df.loc[df[reference_column] == reference, destination_column].values
    return lst

In [82]:
ports = get_list(df_aka, "27-24", "VOYAGE", "PORTS")
ports = ports[0].split(";")
ports

['SURABAYA', 'BALIKPAPAN', 'SAMARINDA', 'BATULICIN']

In [83]:
cost = [get_list(df_fwcost, item, "LOKASI", "COST")[0] for item in ports]
cost

[35000, 24000, 20000, 35000]

In [84]:
trips = get_list(df_aka, "27-24", "VOYAGE", "TRIPS")
trips = trips[0].split(";")
trips


['SURABAYA-BALIKPAPAN',
 'BALIKPAPAN-SAMARINDA',
 'SAMARINDA-BATULICIN',
 'BATULICIN-SURABAYA']

In [85]:
df_ship_distance

,TRIP,DISTANCE
0,AMBON-BIAK,704
1,AMBON-SORONG,234
2,AMBON-SURABAYA,986
3,BALIKPAPAN-BANJARMASIN,333
4,BALIKPAPAN-BITUNG,613
...,...,...
167,BIAK-SERUI,120
168,AMBON-SERUI,810
169,SERUI-AMBON,810
170,BOMBANA-MAKASSAR,343


In [86]:
def get_distances(trips):
    distance = []
    for trip in trips:
        item = get_list(df_ship_distance, trip, "TRIP", "DISTANCE")
        wrapper = list(item)
        distance.append(wrapper)
    return distance
        

In [87]:
distance = get_distances(trips)
distance

[[456], [92], [204], [333]]

In [ ]:
import joblib
import numpy as np
import xgboost as xgb


def predict_freshwater(distances):
    model = joblib.load("../models/modelxgb.pkl")
    scaler = joblib.load("../models/scalerxgb.pkl")
    
    distances = np.array(distances).reshape(1, -1)
    
    if distances.shape[1] < 159:
        padding = 159 - distances.shape[1]
        distances = np.pad(distances, ((0, 0), (0, padding)), mode='constant')
    
    distance_scaled = scaler.transform(distances)
    dmatrix = xgb.DMatrix(distance_scaled)
    
    fw = model.predict(dmatrix)
    return round(fw[0], 2)

In [42]:
distance_numbers = [item[0] for item in distance]
fwneeds = predict_freshwater(distance_numbers)

c:\Users\ilham\miniconda3\envs\vision1\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: training data did not have the following fields: DISTANCE, SHIP_ROUTE_AMBON-BIAK, SHIP_ROUTE_AMBON-SORONG, SHIP_ROUTE_AMBON-SURABAYA, SHIP_ROUTE_BALIKPAPAN-BANJARMASIN, SHIP_ROUTE_BALIKPAPAN-BITUNG, SHIP_ROUTE_BALIKPAPAN-DPL, SHIP_ROUTE_BALIKPAPAN-JAKARTA, SHIP_ROUTE_BALIKPAPAN-PALU, SHIP_ROUTE_BALIKPAPAN-SAMARINDA, SHIP_ROUTE_BALIKPAPAN-SEMARANG, SHIP_ROUTE_BALIKPAPAN-SURABAYA, SHIP_ROUTE_BANJARMASIN-JAKARTA, SHIP_ROUTE_BANJARMASIN-MAKASSAR, SHIP_ROUTE_BANJARMASIN-SAMARINDA, SHIP_ROUTE_BANJARMASIN-SEMARANG, SHIP_ROUTE_BANJARMASIN-SURABAYA, SHIP_ROUTE_BATAM-BELAWAN, SHIP_ROUTE_BATAM-JAKARTA, SHIP_ROUTE_BATULICIN-BERAU, SHIP_ROUTE_BATULICIN-JAKARTA, SHIP_ROUTE_BATULICIN-PALU, SHIP_ROUTE_BATULICIN-SAMPIT, SHIP_ROUTE_BATULICIN-SURABAYA, SHIP_ROUTE_BAUBAU-KENDARI, SHIP_ROUTE_BAUBAU-SURABAYA, SHIP_ROUTE_BELAWAN-BATAM, SHIP_ROUTE_BELAWAN-JAKARTA, SHIP_ROUTE_BELAWAN-KUALATANJUNG, SHIP_ROUTE_BENGKULU-JAKARTA, SHIP_ROUTE_BENGKULU-MERAK, SHIP_ROUTE_BERAU-BITUNG, SHIP_ROUTE_BERAU-SURA

In [66]:
df_fw = pd.DataFrame({
    "Trip": trips,
    "Distance": distance_numbers,
    "Freshwater": fwneeds
})
df_fw

NameError: name 'fwneeds' is not defined

In [161]:
fwneed = predict_freshwater(distance)
fwneed

c:\Users\Afifudin\anaconda3\envs\spil-HRIS\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[17427.57, 5511.14, 22230.39]

In [99]:
import joblib

model = joblib.load("../models/model.pkl")
scaler = joblib.load("../models/scaler.pkl")

In [100]:
distance_scaled = scaler.transform(distance)

c:\Users\Afifudin\anaconda3\envs\spil-HRIS\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [116]:
fw_need = []
for item in distance_scaled:
    fw = model.predict(item.reshape(1, -1))
    fw_need.append(fw[0])

fw_need

[17427.56807881554, 5511.143006085158, 22230.39389783157]

In [2]:
import locale
locale.setlocale(locale.LC_ALL, '')
currency_string = locale.currency(1000000, grouping=True)
currency_string

'Rp1.000.000,00'